## Multi Head Attention

In [79]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


In [80]:
sequence_length = 4  # my name is jay  = 4 tnae kelime var ve sabitlemiş olduk böylece
batch_size = 1
input_dim = 512
d_model = 512
x = torch.randn((batch_size, sequence_length, input_dim))

In [81]:
x.size()

torch.Size([1, 4, 512])

In [82]:
qkv_layer = nn.Linear(input_dim, 3*d_model)
qkv_layer

Linear(in_features=512, out_features=1536, bias=True)

In [83]:
qkv = qkv_layer(x)

In [84]:
qkv.shape

torch.Size([1, 4, 1536])

In [85]:
num_heads = 8
head_dim = d_model//num_heads
qkv = qkv.reshape(batch_size, sequence_length, num_heads, 3*head_dim)

In [86]:
qkv.shape

torch.Size([1, 4, 8, 192])

In [87]:
qkv = qkv.permute(0 ,2, 1, 3) # [batch_size , num_heads, sequence_length, 3*head_dim]
qkv.shape

torch.Size([1, 8, 4, 192])

In [88]:
q, k, v = qkv.chunk(3,dim=-1)
q.shape, k.shape, v.shape

(torch.Size([1, 8, 4, 64]),
 torch.Size([1, 8, 4, 64]),
 torch.Size([1, 8, 4, 64]))

## Self Attention for multiple heads

In [89]:
import math
d_k = q.size()[-1]
scaled = torch.matmul(q, k.transpose(-2,-1)) / math.sqrt(d_k)
scaled.shape

torch.Size([1, 8, 4, 4])

In [90]:
k.T.shape

torch.Size([64, 4, 8, 1])

In [91]:
y = torch.randn(2,3)
print(y)
torch.transpose(y,0,1)


tensor([[-1.8403,  1.0049,  1.0040],
        [ 0.5292,  0.2508,  0.3002]])


tensor([[-1.8403,  0.5292],
        [ 1.0049,  0.2508],
        [ 1.0040,  0.3002]])

In [92]:
torch.transpose(y,1,0)

tensor([[-1.8403,  0.5292],
        [ 1.0049,  0.2508],
        [ 1.0040,  0.3002]])

In [93]:
mask = torch.full(scaled.size(), float("-inf"))
mask = torch.triu(mask, diagonal=1)
mask[0][1]

tensor([[0., -inf, -inf, -inf],
        [0., 0., -inf, -inf],
        [0., 0., 0., -inf],
        [0., 0., 0., 0.]])

In [94]:
a = scaled + mask
(a)[0][0]

tensor([[ 0.3429,    -inf,    -inf,    -inf],
        [-0.0529, -0.1266,    -inf,    -inf],
        [ 0.4176, -0.0893, -0.4129,    -inf],
        [ 0.2425,  0.2139,  0.4821, -0.2128]], grad_fn=<SelectBackward0>)

In [95]:
scaled += mask

In [96]:
np.exp(0.4668) / (np.exp(0.4668)  + np.exp(-0.1652) )   # Aşağıda karşılığı var sonucun aynısı

0.6529428177055896

In [97]:
np.exp(-0.1652) / (np.exp(-0.1652)  + np.exp(0.4668) ) # Aşağıda karşılığı var sonucun aynısı 

0.34705718229441035

In [98]:
scaled += mask


In [99]:
attention = F.softmax(scaled, dim=-1)

In [100]:
attention.shape

torch.Size([1, 8, 4, 4])

In [101]:
attention[0][0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5184, 0.4816, 0.0000, 0.0000],
        [0.4906, 0.2955, 0.2138, 0.0000],
        [0.2579, 0.2507, 0.3278, 0.1636]], grad_fn=<SelectBackward0>)

All the step was included in following function

In [102]:
import math

def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]
    scaled = torch.matmul(q,k.transpose(-1,-2)) / math.sqrt(d_k)
    if mask is not None:
        scaled += mask
    attention = F.softmax(scaled, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention




In [103]:
values, attention = scaled_dot_product(q,k,v,mask=None)


In [104]:
attention.shape

torch.Size([1, 8, 4, 4])

In [105]:
values.shape

torch.Size([1, 8, 4, 64])

In [106]:
attention[0][0]

tensor([[0.3761, 0.2228, 0.2057, 0.1954],
        [0.2347, 0.2180, 0.2550, 0.2923],
        [0.3917, 0.2360, 0.1707, 0.2016],
        [0.2579, 0.2507, 0.3278, 0.1636]], grad_fn=<SelectBackward0>)

In [107]:
values2, attention2 = scaled_dot_product(q,k,v,mask=mask)

In [108]:
attention2[0][0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5184, 0.4816, 0.0000, 0.0000],
        [0.4906, 0.2955, 0.2138, 0.0000],
        [0.2579, 0.2507, 0.3278, 0.1636]], grad_fn=<SelectBackward0>)

In [109]:
values = values.reshape(batch_size, sequence_length, num_heads* head_dim)
values.size()

torch.Size([1, 4, 512])

In [110]:
linear_layer = nn.Linear(d_model, d_model)

In [111]:
out = linear_layer(values)

In [112]:
out.shape


torch.Size([1, 4, 512])

All codes in the class as below code 

In [113]:
import math

def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]
    scaled = torch.matmul(q,k.transpose(-1,-2)) / math.sqrt(d_k)
    if mask is not None:
        scaled += mask
    attention = F.softmax(scaled, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention


class MultiheadAttention(nn.Module):

    def __init__(self, input_dim, d_model, num_heads):
        super().__init__()
        self.input_dim = input_dim
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_layer = nn.Linear(input_dim, 3*d_model)
        self.linear_layer = nn.Linear(d_model, d_model)

    def forward(self, x, mask=None):
        batch_size, sequence_length, input_dim = x.size()
        qkv = self.qkv_layer(x)
        qkv = qkv.reshape(batch_size, sequence_length, self.num_heads,  3*self.head_dim)
        q,k,v = qkv.chunk(3,dim=-1)
        values, attention = scaled_dot_product(q,k,v,mask=mask)
        values = values.reshape(batch_size, sequence_length, self.num_heads * self.head_dim)
        out = self.linear_layer(values)
        return out




Input 

In [120]:
input_dim = 1024
d_model = 512
num_heads = 8

batch_size = 30
sequence_length = 5
x = torch.randn((batch_size, sequence_length, input_dim))


model = MultiheadAttention(input_dim, d_model, num_heads)
out = model.forward(x)

In [121]:
out
out.size()

torch.Size([30, 5, 512])